In [1]:
from gmn_config.evaluation import compute_similarity, auc
from gmn_config.loss import pairwise_loss, triplet_loss
from gmn_config.gmn_utils import *
from gmn_config.configure import *
import numpy as np
import os
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GATConv, dense_mincut_pool
from torch_geometric.utils import to_dense_adj, to_dense_batch
from torch_geometric.data import Data
from torch_geometric.utils import dense_to_sparse
import torch
from torch_geometric.nn import dense_mincut_pool
from torch_geometric.utils import to_dense_adj, to_dense_batch
from gnn_models.gat import *
import numpy as np
from sklearn.decomposition import PCA
import torch

In [2]:
# dataset

dataset = Planetoid(root='./data', name='Cora')
data = dataset[0]

In [3]:
model = GATwithMinCutPooling(num_features=dataset.num_features, num_classes=dataset.num_classes, dropout=0.6)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)    

for epoch in range(50):
    train_loss, train_class_loss, train_mincut_loss, train_ortho_loss = train(model, data, optimizer, dataset)
             
test_loss, test_class_loss, test_mincut_loss, test_ortho_loss = test(model, data, dataset)
print(f"Test Loss: {test_loss}")
print(f'\ttest_class_loss = {test_class_loss}, test_mincut_loss = {test_mincut_loss}, test_ortho_loss = {test_ortho_loss}') 


Test Loss: 0.7293213605880737
	test_class_loss = 0.6160595417022705, test_mincut_loss = -0.999330997467041, test_ortho_loss = 1.1125928163528442


In [4]:
model.eval()
with torch.no_grad():
    x, x_pool, adj_pool, mincut_loss, ortho_loss = model(data, dataset.num_classes)

edge_index_pool = dense_to_sparse(adj_pool)[0]

pooled_data = Data(x=x_pool, edge_index=edge_index_pool)

In [5]:
g1 = Data(x=data.x, edge_index=data.edge_index)
g2 = pooled_data

In [6]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

# Print configure
config = get_default_config()
for k, v in config.items():
    print("%s= %s" % (k, v))

# Set random seeds
seed = config["seed"]
random.seed(seed)
np.random.seed(seed + 1)
torch.manual_seed(seed + 2)
torch.backends.cudnn.deterministic = False
torch.backends.cudnn.benchmark = True
torch.autograd.set_detect_anomaly(True)


training_set, validation_set = build_datasets(config)

if config["training"]["mode"] == "pair":
    training_data_iter = training_set.pairs(config["training"]["batch_size"])
    first_batch_graphs, _ = next(training_data_iter)
else:
    training_data_iter = training_set.triplets(config["training"]["batch_size"])
    first_batch_graphs = next(training_data_iter)

node_feature_dim = first_batch_graphs.node_features.shape[-1]
edge_feature_dim = first_batch_graphs.edge_features.shape[-1]

model, optimizer = build_model(config, node_feature_dim, edge_feature_dim)
model.to(device)

encoder= {'node_hidden_sizes': [32], 'node_feature_dim': 1, 'edge_hidden_sizes': [16]}
aggregator= {'node_hidden_sizes': [128], 'graph_transform_sizes': [128], 'input_size': [32], 'gated': True, 'aggregation_type': 'sum'}
graph_embedding_net= {'node_state_dim': 32, 'edge_state_dim': 16, 'edge_hidden_sizes': [64, 64], 'node_hidden_sizes': [64], 'n_prop_layers': 5, 'share_prop_params': True, 'edge_net_init_scale': 0.1, 'node_update_type': 'gru', 'use_reverse_direction': True, 'reverse_dir_param_different': False, 'layer_norm': False, 'prop_type': 'matching'}
graph_matching_net= {'node_state_dim': 32, 'edge_state_dim': 16, 'edge_hidden_sizes': [64, 64], 'node_hidden_sizes': [64], 'n_prop_layers': 5, 'share_prop_params': True, 'edge_net_init_scale': 0.1, 'node_update_type': 'gru', 'use_reverse_direction': True, 'reverse_dir_param_different': False, 'layer_norm': False, 'prop_type': 'matching', 'similarity': 'dotproduct'}
model_type= matching
data= {'problem': 'graph_edit_distance', 'datase

GraphMatchingNet(
  (_encoder): GraphEncoder(
    (MLP1): Sequential(
      (0): Linear(in_features=8, out_features=32, bias=True)
    )
    (MLP2): Sequential(
      (0): Linear(in_features=4, out_features=16, bias=True)
    )
  )
  (_aggregator): GraphAggregator(
    (MLP1): Sequential(
      (0): Linear(in_features=32, out_features=256, bias=True)
    )
    (MLP2): Sequential(
      (0): Linear(in_features=128, out_features=128, bias=True)
    )
  )
  (_prop_layers): ModuleList(
    (0-4): 5 x GraphPropMatchingLayer(
      (_message_net): Sequential(
        (0): Linear(in_features=80, out_features=64, bias=True)
        (1): ReLU()
        (2): Linear(in_features=64, out_features=64, bias=True)
      )
      (_reverse_message_net): Sequential(
        (0): Linear(in_features=80, out_features=64, bias=True)
        (1): ReLU()
        (2): Linear(in_features=64, out_features=64, bias=True)
      )
      (GRU): GRU(96, 32)
    )
  )
)

In [13]:
def convert_graph(graph):
    pca = PCA(n_components=8)
    reduced_node_features = pca.fit_transform(graph.x[1])
    
    from_idx = graph.edge_index[0]
    to_idx = graph.edge_index[1]
    
    num_edges = graph.edge_index.shape[1]
    graph_idx = np.zeros(graph.x.shape[0], dtype=int)
    
    node_features = torch.tensor(reduced_node_features, dtype=torch.float32)
    edge_features = torch.ones((num_edges, 4), dtype=torch.float32)
    
    from_idx = torch.tensor(from_idx, dtype=torch.long)
    to_idx = torch.tensor(to_idx, dtype=torch.long)
    graph_idx = torch.tensor(graph_idx, dtype=torch.long)
    
    return node_features, edge_features, from_idx, to_idx, graph_idx

In [12]:
g2

Data(x=[7, 1433], edge_index=[2, 42])

In [14]:
g1_node_features, g1_edge_features, g1_from_idx, g1_to_idx, g1_graph_idx = convert_graph(g1)
g2_node_features, g2_edge_features, g2_from_idx, g2_to_idx, g2_graph_idx = convert_graph(g2)

model.eval()

stacked_node_features = torch.cat((g1_node_features, g2_node_features))
stacked_edge_features = torch.cat((g1_edge_features, g2_edge_features))

stacked_from_idx = torch.cat((g1_from_idx, g2_from_idx+g2.x.shape[0]))
stacked_to_idx = torch.cat((g1_to_idx, g2_to_idx+g2.x.shape[0]))

stacked_graph_idx = torch.cat((torch.tensor(np.zeros(g1.x.shape[0], dtype=int)), torch.tensor(np.ones(g2.x.shape[0], dtype=int))))

graph_vectors = model(
        stacked_node_features.to(device),
        stacked_edge_features.to(device),
        stacked_from_idx.to(device), 
        stacked_to_idx.to(device),
        stacked_graph_idx.to(device),
        2
    )

x, y = reshape_and_split_tensor(graph_vectors, 2)
similarity = compute_similarity(config, x, y)
print(similarity)

ValueError: Expected 2D array, got 1D array instead:
array=[0. 0. 0. ... 0. 0. 0.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.